In [10]:
# 🌴 Tropical Trendsetters: Data Wrangling Notebook for Hurricane Path and Intensity Analysis 🌊

# Welcome to the **Tropical Trendsetters** data cleaning notebook! In this project, we’re working to detect trends in florida's hurricanes. 

# We're wrangling 2 datasets today. 
# 1. Hurricane path data with name, lat, lon, wind, and pressure readings. 
# - --- we're going to rename a column and clean a column so every date is in the same format 
# 2. water table data 
# --- --- we're going to filter this down to a specific date 

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
original_df = pd.read_csv("GEO557Tropical_Storm_Dataset.csv") 
#
print(original_df.info())
original_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               2240 non-null   int64  
 1   Name               2240 non-null   object 
 2   BASIN              0 non-null      float64
 3   ISO_TIME_________  2240 non-null   object 
 4   NATURE             2240 non-null   object 
 5   LAT                2240 non-null   float64
 6   LON                2240 non-null   float64
 7   WMO WIND           1180 non-null   float64
 8   WMO PRES           1180 non-null   float64
 9   USA WIND           2240 non-null   int64  
 10  USA PRES           2240 non-null   int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 192.6+ KB
None


,Year,Name,BASIN,ISO_TIME_________,NATURE,LAT,LON,WMO WIND,WMO PRES,USA WIND,USA PRES
0,2023,IDALIA,NaN,2023-08-26 12:00:00,TS,20.8,-86.1,25.0,1006.0,25,1006
1,2023,IDALIA,NaN,15:00:00,TS,21.1,-86.1,NaN,NaN,25,1006
2,2023,IDALIA,NaN,18:00:00,TS,21.3,-86.2,25.0,1006.0,25,1006
3,2023,IDALIA,NaN,21:00:00,TS,21.3,-86.3,NaN,NaN,28,1005
4,2023,IDALIA,NaN,2023-08-27 0:00:00,TS,21.1,-86.4,30.0,1004.0,30,1004
5,2023,IDALIA,NaN,3:00:00,TS,20.8,-86.7,NaN,NaN,30,1003
6,2023,IDALIA,NaN,6:00:00,TS,20.5,-86.8,30.0,1002.0,30,1002
7,2023,IDALIA,NaN,9:00:00,TS,20.2,-86.6,NaN,NaN,33,1001
8,2023,IDALIA,NaN,12:00:00,TS,19.9,-86.3,35.0,999.0,35,999
9,2023,IDALIA,NaN,15:00:00,TS,19.9,-86.0,NaN,NaN,38,998


In [20]:
def populate_full_dates(df): 
    #iterate through DF and fix dates 
    # Initialize variable to hold the last full date encountered
    current_date = None
    
    # Iterate through the ISO_TIME column and update times based on the last full date
    for i, iso_time in enumerate(df['ISO_TIME']):
        if len(iso_time) > 8:  # Full datetime (YYYY-MM-DD HH:MM:SS)
            # Set current_date to the full date part of the timestamp
            current_date = iso_time[:10]  # Extract the date portion (YYYY-MM-DD)
        else:
            # If only time is present, add the current_date to create a full timestamp
            df.at[i, 'ISO_TIME'] = f"{current_date} {iso_time}"
    
    # Convert ISO_TIME column to datetime for consistency
    df['ISO_TIME'] = pd.to_datetime(df['ISO_TIME'])
    return df 

# originally the data was gatherd from 23 different websites from NOAA historical hurricane tracks   https://bit.ly/3NNylR4
# the name and year columns I added as I gathered the data. 

# some issues with the dataset involve ISO time being seuqential, so the first one we're going to wrangle is  
# ISO_TIME____ column, the name and the data both need help. 
# YYYY-MM-DD but every other measurement in that section doesn't have that until it hits the next day 
# we're going to do the following 3 things, 
# 1. rename the iso_time column 
# 2. add dates to match the TIME
df = pd.read_csv("GEO557Tropical_Storm_Dataset.csv") 
# Step 1: Rename the ISO_TIME_________ column to ISO_TIME
df.rename(columns={'ISO_TIME_________': 'ISO_TIME'}, inplace=True)
df.head(5) 

# Step 2: we have to iterate through the data set, and if ISO_TIME has a full date in it, pull that out, and populate until it finds a new Full date 
df = populate_full_dates(df)

print(df.info())
df.head(10)
# notice the ISO_TIME column is uniform the year is paired with the timestamp. this will make our lives easier in the future. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Year      2240 non-null   int64         
 1   Name      2240 non-null   object        
 2   BASIN     0 non-null      float64       
 3   ISO_TIME  2240 non-null   datetime64[ns]
 4   NATURE    2240 non-null   object        
 5   LAT       2240 non-null   float64       
 6   LON       2240 non-null   float64       
 7   WMO WIND  1180 non-null   float64       
 8   WMO PRES  1180 non-null   float64       
 9   USA WIND  2240 non-null   int64         
 10  USA PRES  2240 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(2)
memory usage: 192.6+ KB
None


,Year,Name,BASIN,ISO_TIME,NATURE,LAT,LON,WMO WIND,WMO PRES,USA WIND,USA PRES
0,2023,IDALIA,NaN,2023-08-26 12:00:00,TS,20.8,-86.1,25.0,1006.0,25,1006
1,2023,IDALIA,NaN,2023-08-26 15:00:00,TS,21.1,-86.1,NaN,NaN,25,1006
2,2023,IDALIA,NaN,2023-08-26 18:00:00,TS,21.3,-86.2,25.0,1006.0,25,1006
3,2023,IDALIA,NaN,2023-08-26 21:00:00,TS,21.3,-86.3,NaN,NaN,28,1005
4,2023,IDALIA,NaN,2023-08-27 00:00:00,TS,21.1,-86.4,30.0,1004.0,30,1004
5,2023,IDALIA,NaN,2023-08-27 03:00:00,TS,20.8,-86.7,NaN,NaN,30,1003
6,2023,IDALIA,NaN,2023-08-27 06:00:00,TS,20.5,-86.8,30.0,1002.0,30,1002
7,2023,IDALIA,NaN,2023-08-27 09:00:00,TS,20.2,-86.6,NaN,NaN,33,1001
8,2023,IDALIA,NaN,2023-08-27 12:00:00,TS,19.9,-86.3,35.0,999.0,35,999
9,2023,IDALIA,NaN,2023-08-27 15:00:00,TS,19.9,-86.0,NaN,NaN,38,998


In [13]:
# a second dataset we needed to wrangle was ground water measurement levels from the various stations in florida. 
# we need to use the most recent sample before the target storm date. 

# Read the file
water_table_df = pd.read_csv("TampaBayGW20232024.txt", sep='\t')

# Display the number of distinct stations so we know if we have one value for each station.
distinct_station_count = water_table_df['StationID'].nunique()
print("Number of distinct stations:", distinct_station_count)
print("original df len =" + str(len(water_table_df)))

print(water_table_df.info())
water_table_df.head(5)


Number of distinct stations: 169
original df len =72808
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72808 entries, 0 to 72807
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   WBodyID         72808 non-null  int64  
 1   WaterBodyName   72808 non-null  object 
 2   DataSource      72808 non-null  object 
 3   StationID       72808 non-null  int64  
 4   StationName     72808 non-null  object 
 5   Latitude_DD     72808 non-null  float64
 6   Longitude_DD    72808 non-null  float64
 7   ReachCode       0 non-null      float64
 8   POR_Min         0 non-null      float64
 9   POR_Max         0 non-null      float64
 10  SampleDate      72808 non-null  object 
 11  Characteristic  72808 non-null  object 
 12  Parameter       72808 non-null  object 
 13  ResultValue     72808 non-null  float64
dtypes: float64(6), int64(2), object(6)
memory usage: 7.8+ MB
None


,WBodyID,WaterBodyName,DataSource,StationID,StationName,Latitude_DD,Longitude_DD,ReachCode,POR_Min,POR_Max,SampleDate,Characteristic,Parameter,ResultValue
0,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,670753,115th STREET FLDN,28.052844,-82.461097,NaN,NaN,NaN,9/1/2023 12:00:00 AM,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,23.21
1,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,670753,115th STREET FLDN,28.052844,-82.461097,NaN,NaN,NaN,9/2/2023 12:00:00 AM,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,23.23
2,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,670753,115th STREET FLDN,28.052844,-82.461097,NaN,NaN,NaN,9/3/2023 12:00:00 AM,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,23.19
3,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,670753,115th STREET FLDN,28.052844,-82.461097,NaN,NaN,NaN,9/4/2023 12:00:00 AM,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,23.16
4,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,670753,115th STREET FLDN,28.052844,-82.461097,NaN,NaN,NaN,9/5/2023 12:00:00 AM,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,23.13


In [17]:

# Define the function to filter 
def filter_closest_date_per_station(df, storm_date='2023-10-01'):
    # Convert SampleDate column to datetime with a specified format to avoid warnings
    df['SampleDate'] = pd.to_datetime(df['SampleDate'], format='%m/%d/%Y %I:%M:%S %p')
    
    # Convert storm_date to datetime format
    storm_date = pd.to_datetime(storm_date)
    df_before_storm = df[df['SampleDate'] <= storm_date]
    
    # Sort the filtered DataFrame by StationID and SampleDate in descending order
    df_before_storm = df_before_storm.sort_values(['StationID', 'SampleDate'], ascending=[True, False])
    
    # For each station, keep only the first row
    closest_date_df = df_before_storm.groupby('StationID').head(1)
    
    return closest_date_df

#SET STORM DATE EX: '2023-10-01'
storm_date ='2023-10-01'
filtered_df = filter_closest_date_per_station(water_table_df, storm_date=storm_date)

# this should have only 169 values one for each station. 
filtered_df.shape

output_file_name = storm_date + "_water_table.csv"

#we expect to only have 169 data points now, one for each station. and the sample date column is on the target date which is exactly what we want! 
filtered_df.to_csv(output_file_name, index=False)
print("new filtered dataframe len = " + str(len(filtered_df)))

filtered_df.info()
filtered_df.head()

new filtered dataframe len = 169
<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 40846 to 26294
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   WBodyID         169 non-null    int64         
 1   WaterBodyName   169 non-null    object        
 2   DataSource      169 non-null    object        
 3   StationID       169 non-null    int64         
 4   StationName     169 non-null    object        
 5   Latitude_DD     169 non-null    float64       
 6   Longitude_DD    169 non-null    float64       
 7   ReachCode       0 non-null      float64       
 8   POR_Min         0 non-null      float64       
 9   POR_Max         0 non-null      float64       
 10  SampleDate      169 non-null    datetime64[ns]
 11  Characteristic  169 non-null    object        
 12  Parameter       169 non-null    object        
 13  ResultValue     169 non-null    float64       
dtypes: datetime64[ns](1), fl

,WBodyID,WaterBodyName,DataSource,StationID,StationName,Latitude_DD,Longitude_DD,ReachCode,POR_Min,POR_Max,SampleDate,Characteristic,Parameter,ResultValue
40846,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,17769,ROMP 61 U FLDN AQ MONITOR,27.908528,-82.161194,NaN,NaN,NaN,2023-10-01,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,38.25
27320,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,17822,ROMP 123 HTRN AS/U FLDN AQ MONITOR,27.675472,-82.251306,NaN,NaN,NaN,2023-10-01,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,16.21
34090,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,17837,ROMP 48 HTRN AS MONITOR,27.740889,-82.142556,NaN,NaN,NaN,2023-10-01,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,88.76
34514,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,17840,ROMP 48 U FLDN AQ (TMPA/SWNN) MONITOR,27.740861,-82.142556,NaN,NaN,NaN,2023-10-01,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,41.63
34938,-9999,"Weather, Rainfall, Ground Water",SWFWMD_HYDRO,17841,ROMP 48 U FLDN AQ MONITOR (AVPK),27.740889,-82.142500,NaN,NaN,NaN,2023-10-01,"Elevation, water surface (NAVD88)",Level_ft_NAVD88,38.19
